In [13]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Beispiel-Implementierung: Lokale Suchmaschine

## Ziel der Beispiel-Implementierung
Im Folgenden wird eine Beispiel-Implementierung der zuvor theoretisch diskutierten Inhalte
vorgestellt. Dabei wird eine lokale Suchmaschine entwickelt, welche in der Lage ist,
PDF-Dateien auf einem lokalen Computer-System zu parsen, in einen invertierten Index
aufzunehmen, sowie Suchanfragen eines Benutzers sinnvoll zu beantworten. Zur Relevanz-
Bestimmung der Dokumente wird das TF-IDF-Maß, welches bereits vorgestellt wurde,
genutzt. Das Speichern des Index wird mit der von Python mitgelieferte Datenstruktur Dictionary,
welche im Grunde eine Hashmap ist, umgesetzt. Weiter werden Bibliotheken eingesetzt,
welche einige Vorarbeit leisten und damit den Code der Beispiel-Implementierung
auf das Wesentliche beschränken. Die Anwendung soll die grundlegende Arbeitsweise eines
Information Retrieval-Systems darlegen.

## Genutzte Bibliotheken
Vor der eigentlichen Implementierung der lokalen Suchmaschine werden einige Module eingebunden, welche die Implementierung unterstützen. Folgend werden die Module aufgelistet und wichtige Module im jeweiligen Abschnitt genauer erläutert:
- Apache Tika: Erkennt und extrahiert Metadaten und Texte aus über tausend Dateitypen
- Math: Standard Python-Modul für mathematische Funktionen
- OS: Stellt Betriebssystem-Funktionalitäten bereit. Wird hier genutzt um durch Verzeichnisse zu navigieren
- Filetype: Wird zur Dateityp-Erkennung genutzt
- re: Das re-Modul stellt Funktionen bereit, mit denen mit regulären Ausdrücken gearbeitet werden kann. In der Beispiel-Implementierung werden mit diesem Modul die Tokens ermittelt
- Platform: Prüfung, auf welchem Betriebssystem das Programm läuft, da Windows- und Linux-Systeme verschiedene Dateisysteme nutzen
- Operator: Dieses Modul exportiert effiziente Funktionen, die den eigentlichen Operatoren von Python entsprechen. Es findet nur Anwendung in der Sortierung der zu zurückgebenden Dokumente in der retrieve-Methode.
- NLTK: Stellt Funktionen zur Verfügung, die es ermöglichen mit menschlichen Sprachdaten zu arbeiten.

### Apache Tika
Bei Apache Tika handelt es sich um ein Framework um Inhalte aus Dateien zu erkennen und zu analysieren. Es ist in der Lage Text und Metadaten aus über tausend verschiedenen Arten von Dateien zu extrahieren. Tika liefert einen Parser, mit dessen Hilfe der Text aus - unter anderem - PDF-Dateien extrahiert werden kann. Mit dem Aufruf <i>parser.from_file(file)</i> kann eine PDF-Datei in reinen Text umgewandelt werden. Die Funktion liefert ein Dictionary zurück, welches einen Key content besitzt, über den auf den Inhalt der PDF-Datei zugegriffen werden kann.

### filetype
Mittels filetype ist es möglich, unabhängig von der Dateiendung, den Typ einer Datei zu ermitteln. Dies hat den Vorteil, dass die Suchmaschine sowohl unter Windows, als auch unter Unix-Systemen, alle PDF-Dateien finden kann, da unter Unix die Dateiendung keine garantierten Rückschlüsse auf den Typ der Datei zulässt.

### NLTK
NLTK (natural language toolkit) ist eine Bibliothek für Python, die für die Verarbeitung natürlicher Sprachen eingesetzt wird. Dabei bietet die Bibliotheken Funktionen für unter anderem Textklassifikation, Tokenization und Stemming. In diesem Beispiel wird nltk verwendet, um die Eingabetexte der Dokumente und die Eingaben des Nutzers zu Tokens zu verarbeiten. Dazu wird eine Klasse verwendet, die auf Basis von Regular Expressions arbeitet. Mehr dazu wird anhand der Implementierung gezeigt.

### Platform
Das Platform-Modul wird in dieser Implementierung dazu verwendet, festzustellen, auf welchem Betriebssystem die lokale Suchmaschine ausgeführt wird. Dies muss ermittelt werden, da auf Windows und Linux unterschiedliche Dateisysteme arbeiten. In Linux ist das Startverzeichnis immer das Root-Verzeichnis („/“). Unter Windows gibt meist mehrere Partitionen, die alle durchsucht werden müssen.

In [14]:
from tika import parser
import filetype
import math
import os
import string
import platform
import operator
import re
from nltk.tokenize import RegexpTokenizer

## Die Document-Klasse
Das Speichern der für das Retrieval wichtigen Informationen, geschieht mittels einer Document-Klasse. Diese Klasse hält alle Member-Variablen, die wichtig sind, um das TF-IDF-Maß berechnen zu können. Diese Member-Variablen sind:
- url: String mit dem Pfad zum Dokument, welches von dieser Instanz repräsentiert wird
- length: Integer, welcher die Anzahl der Wörter in diesem Dokument darstellt
- docId: Integer, welcher dem Dokument einen eindeutigen Identifikator gibt
- score: Float, welcher die Gewichtung der Dokument-Instanz zu einer Anfrage angibt
- termList: Eine Liste von Strings, die die Terme des Dokumentes entsprechen. Diese wird durch die Methode _preprocess erzeugt

Die Member-Variable <i>url</i> soll am Ende des Retrievalprozesses zurückgegeben werden, da der Nutzer durch den Pfad direkten Zugriff auf das Dokument bekommt. Die Member-Variablen <i>length</i>, <i>termList</i> und <i>score</i> werden für die Berechnung des TF-IDF-Maßes benötigt und so auch für den Retrievalprozess. Die Member-Variable <i>docId</i> wird in der Indexklasse einerseits genutzt, um die Identifier in dem <i>invIndex</i>-Dictionary den jeweiligen Termen zuzuordnen. Des Weiteren erfolgt die Zuordnung der jeweiligen ID zu ihrer Document-Instanz über die <i>docHashmap</i> in der Indexklasse.

In [15]:
class Document:
    def __init__(self, url, length, docId, termList):
        self.url = url
        self.length = length
        self.docId = docId
        self.score = 0.
        self.termList = termList

### TF-IDF
Die Document-Klasse hält neben den benötigten Attributen auch die Scoringmethode <i>tf_idf</i>. Dies ist die Implementierung des TF-IDF-Maßes und berechnet für jedes Dokument die Gewichtung für eine gegebene Suchquery aus. Hierbei benötigt die Funktion die Terme der Suchquery, welche über das Attribut terms übergeben werden. Das Attribut <i>df</i> steht für die Document Frequency, welches für jeden Term die Anzahl der gefunden Dokumente in Form eines Dictionaries beinhaltet. Über das Attribut <i>fileCount</i> wird die Anzahl der Dokumente in der Kollektion, hier die Dateien die im invertierten Index aufgenommen wurden, übergeben.

Als erstes wird das Dictionary <i>tfDict</i> für die Term Frequency erstellt und mit den Termen der Query als Schlüssel und den Werten 0 initialisiert. In der nächsten For-Schleife wird über <i>self.termList</i> iteriert, welches die Liste der Terme des Dokumentes sind. Wenn ein Term auch in der Suchquery enthalten ist, also <i>if term in terms</i>, dann wird im <i>tfDict</i> der Wert des gefunden Terms um eins aufaddiert. Am Ende enthält das <i>tfDict</i> die Terme der Suchquery als Schlüssel und die Anzahl ihrer Vorkommnisse im Dokument als Wert, also die Term Frequency. Als letztes muss jede Term Frequency mit der Inverse Document Frequency multipliziert werden. Dafür wird über das Dictionary <i>df</i> iteriert und für jedes Key-Value-Paar die Inverse Document Frequency ausgerechnet und auf die jeweilige Term Frequency multipliziert. Am Ende wird die Summe aller TF-IDF-Maße der <i>score</i>-Variable zugeordnet und bilden so die finale Gewichtung für die gegebene Suche.

In [16]:
def tf_idf(self, terms, df, fileCount):
    
    tfDict = {}
    for term in terms:
        tfDict[term] = 0
    
    ind = Index()
        
    for term in self.termList:
        if term in terms:
            tfDict[term] = tfDict[term]+1

    for key, value in df.items():
        idf = math.log((ind.fileCount/value+1),10)
        tfDict[key] = tfDict[key]*idf
    
    self.score = sum(tfDict.values())

Document.tf_idf = tf_idf

## Der Index
Die Index-Klasse beinhaltet alle Methoden um den invertierten Index aufzubauen und die <i>retrieve</i>-Methode, welche die Dokumente, sortiert nach deren Gewicht, zu einer gegebenen Query zurückgibt. Folgenden Member-Variablen werden für den invertierten Indexaufbau und die <i>retrieve</i>-Methode benötigt:

- invIndex: Ist ein Dictionary mit einem Term als Schlüssel und einer Menge von Document IDs als Wert
- fileCount: Zählt beim Aufbau des invertierten Indexes die Dokumente, die in diesem aufgenommen werden
- docHashmap: Ist ein Dictionary welches eine Document ID ihrer zugehörige Document-Klasseninstanz zuordnet

Die Member-Variable <i>invIndex</i> ist der invertierte Index welcher bei der retrieve-Methode die Document IDs zu einen gegebenen Term aus der Suchquery zurückgibt. Des Weiteren wird die Member-Variable <i>docHashmap</i> dafür genutzt, um über die Document IDs auf die jeweiligen Dokumentinstanzen zuzugreifen und so auf ihren Inhalt und die Scoringfunktion <i>tf_idf</i>. Die Member-Variable <i>fileCount</i> gibt die Größe der Kollektion wieder und wird bei der Berechnung des TF-IDF-Maßes benötigt.

In [17]:
class Index:
    
    def __init__(self):
        self.invIndex = {}
        self.fileCount = 0
        self.docHashmap = {}

### buildIndex
Diese Methode buildIndex baut den invertierten Index auf und nutzt dafür die <i>\_getStartDirectories</i>- und die <i>\_addToIndex</i>-Methode. Dabei kann der <i>buildIndex</i>-Methode optional Startverzeichnisse in Form einer Liste mit raw Strings mitgegeben werden. In der ersten If-Abfrage wird geprüft, ob die Liste <i>startDirectories</i> leer ist. Dies ist der Fall, wenn der Nutzer keine Startverzeichnisse mitgegeben hat. Dann werden die Startverzeichnisse mithilfe der <i>\_getStartDirectories</i>-Methode ermittelt. 

Der erste Schritt stellt das Iterieren über alle Startverzeichnisse der Liste <i>startDirectories</i> dar. Anschließend werden für das Startverzeichnis, und alle darunterliegenden Verzeichnisse, bis zur untersten Ebene, die Dateien mithilfe der <i>os.walk</i>-Funktion geholt. Für jede Datei wird dann mit den Funktionen <i>os.path.abspath</i> und <i>os.path.join</i> der absolute Pfad gebildet. Danach wird durch das Modul <i>filetype</i> ermittelt, ob es sich um ein PDF-Dokument handelt. Ist ein Dokument vom Typ PDF, wird mithilfe des <i>Tika</i>-Moduls, genauer gesagt mit dem <i>parser</i>, der Text aus dem PDF-Dokument extrahiert. Dies geschieht in dem der Funktion <i>from_file</i> des Parsers mit dem Pfad der Datei aufgerufen wird. Bei der Rückgabe kann mithilfe des Schlüssels <i>content</i> auf den Text zugegriffen werden, welcher dann in der Variable
<i>rawText</i> gespeichert wird.

Für jede entdeckte PDF-Datei wird die Member-Variable <i>fileCount</i> um eins erhöht. Da die Zahl sich bei jedem gefunden Dokument ändert, wird diese gleich als Document ID genutzt, da sie für jedes Dokument eindeutig ist. Im Folgenden Schritt wird der <i>rawText</i> mithilfe der <i>\_preprocessText</i>-Methode normalisiert und in eine Liste von Tokens geteilt und in der Liste
<i>processedText</i> gespeichert. Nun sind alle Daten vorhanden um eine Documentinstanz zu erstellen. Diese enthält den Pfad zum Dokument, die Anzahl der Wörter, die Document ID und den Text als Liste von Tokens.

Im vorletztem Schritt wird die erstellte Documentinstanz seiner Document ID in der Member-Variable <i>docHashmap</i> zugeordnet, um später auf diese Instanz zurückgreifen zu können. Als letztes wird der invertierte Index, mithilfe der <i>\_addToIndex-Methode</i>, der Tokenliste <i>processedText</i> und der Document ID, aktualisiert.

In [18]:
def buildIndex(self, startDirectories=[]):
    
    if not startDirectories:
        startDirectories = self._getStartDirectories()
        
    for directory in startDirectories:
        for root, _, files in os.walk(directory):
            for file in files:
                
                path = os.path.abspath(os.path.join(root, file))
                
                try:
                    if filetype.guess(path).mime == 'application/pdf':

                        fileData = parser.from_file(path)
                        rawText = fileData['content']
                        self.fileCount += 1
                    
                        processedText = self._preprocessText(rawText)
                        document = Document(path, len(processedText), self.fileCount, processedText)
                        self.docHashmap.update({self.fileCount : document})
                        self._addToIndex(self.fileCount, processedText)
                except:
                    print(end="")
                    continue

Index.buildIndex = buildIndex

### Hilfsmethoden
In diesem Abschnitt werden die drei Hilfsmethoden <i>\_getStartDirectories</i>, <i>\_preprocessText</i> und <i>\_addToIndex</i> vorgestellt, welche in der <i>buildIndex</i>-Methode genutzt werden.

#### \_getStartDirectories
Die Methode <i>\_getSartDirectories</i> liefert eine Liste der Start-Verzeichnisse, abhängig vom Betriebssystem auf dem die Suchmaschine läuft. In diesen Verzeichnissen werden rekursiv nach PDF-Dateien gesucht, welche in den Index mit einfließen. Falls das zugrunde liegende Betriebssystem ein Linux-basiertes oder Mac-basiertes System ist, wird die Liste <i>["/"]</i> zurückgegeben, da das Verzeichnis / immer das Root-Verzeichnis ist. Bei einem auf Windows basierenden Systemen gibt es wiederum mehrere Partitionen, welche immer mit einem Großbuchstaben abgekürzt werden. Dementsprechend gibt es unter Windows auch mehrere Root-Verzeichnisse.

Zuerst wird mithilfe der <i>system</i>-Funktion des <i>platform</i>-Moduls geprüft welches Betriebssystem vorliegt. Bei einem auf Linux- (Rückgabe "Linux") oder Mac-basierenden System (Rückgabe "Darwin") wird einfach eine List mit dem Element "/" erstellt.

Bei einem auf Windows basierenden Betriebssystem (Rückgabe "Windows") ist das Erstellen der Startverzeichnisse aufwändiger. Hierbei werden alle Großbuchstaben, im Code durch <i>string.ascii\_uppercase</i> aufrufbar, darauf geprüft eine Partition zu sein. Dies wird mithilfe der <i>os.path.exists</i>-Methode realisiert. Ist ein Großbuchstabe tatsächliche eine Partition auf dem Computer, so wird er in der Liste gespeichert. Jedoch wird an den Großbuchstaben noch der String <i>":\"</i> angehangen, damit die buildIndex-Methode mit den Elementen als Start-Verzeichnisse arbeiten kann.

In [19]:
def _getStartDirectories(self):

    if platform.system() == "Linux":
        directories = ["/"]
        
    elif platform.system() == "Darwin":
        directories = ["/"]
        
    elif platform.system() == "Windows":
        directories = ['%s:\\' % d for d in string.ascii_uppercase if os.path.exists('%s:' % d)]
        
    else:
        raise EnvironmentError
        
    return directories

Index._getStartDirectories = _getStartDirectories

#### \_addToIndex

Die Methode <i>\_addToIndex</i> soll die Dokumenten ID zu den invertierten Index der in dem Dokument vorkommenden Terme hinzufügen. Hierfür bekommt die Methode eine Liste von Termen die in einem PDF-Dokument vorkommen über den Parameter <i>terms</i> und die zum Dokumente gehörige Document ID als Parameter <i>documentID</i> übergeben.

Für jeden Term in der Liste <i>terms</i> wird dazu der zum Term gehörige Eintrag im invertierten Index nachgeschlagen. Schlägt der Versuch fehl, da noch kein Eintrag des Terms in dem invertierten Index vorhanden ist, wird ein neuer Eintrag für diesen Term und der übergebenen Document ID erstellt. Wird ein Eintrag gefunden, wird die übergebende Document ID der schon vorhandene Menge hinzugefügt und der invertierte Index wird geupdatet.

In [20]:
def _addToIndex(self, documentID, terms):
    
    for term in terms:
        
        try:
            docSet = self.invIndex[term]
            docSet.add(documentID)
            self.invIndex.update({term : docSet})
            
        except KeyError:
            docSet = {documentID}
            self.invIndex.update({term : docSet})
    
Index._addToIndex = _addToIndex

#### _preprocessText
Diese Methode dient der Vorverarbeitung der Texte, die in den pdf-Dokumenten stehen. Hierfür wird der Text eines PDF-Dokumentes an den Parameter _text_ übergeben. Als erster Schritt wird der gesamte Text in Lower-Case (Kleinschreibung) gesetzt, damit später bei der Suche die Groß- bzw. Kleinschreibung irrelevant ist. Das Ergebnis wird in der Variable _lowerText_ gespeichert.
Im nächsten Schritt werden alle Zahlen aus _lowerText_ entfernt und in _prepText_ gespeichert, da Zahlen für die Textsuche nicht von Bedeutung sind.

Als nächstes wird mithilfe der Klasse _RegexpTokenizer_, die durch die nltk-Bibliothek zur Verfügung gestellt wird, der String _prepText_ in eine Liste von Tokens aufgespalten. Was als Token gewertet wird, wird mithilfe einer _regular expression_ definiert, im Deutschen regulärer Ausdruck genannt. Ein regulärer Ausdruck ist eine Zeichenkette, welche eine Menge von bestimmten Zeichenketten beschreibt. Der gewünschte reguläre Ausdruck wird dem Konstruktor der _RegexpTokenizer_-Klasse in Form eines raw Strings übergeben. Ein raw String ist ein String, welcher mit einem _r_ am Anfang gekennzeichnet ist und ein Backslash (\\) als ein Literal behandelt und nicht als ein Escape-Zeichen. Dies ist bei regulären Ausdrücken nützlich, da in diesen viel mit Backslashes gearbeitet wird.

Im Folgenden wird der raw String bzw. reguläre Ausdruck näher betrachtet, um zu verstehen, was als ein Token gewertet wird. Der erste Teil des regulären Ausdrucks _[a-zA-Z]+-\$_ definiert alle Buchstabenketten mit einen oder mehreren Elementen, die mit einem Bindestrich enden, als Token. Die eckigen Klammern werden bei regulären Ausdrücken genutzt, um eine Zeichenauswahl zu definieren. Das bedeutet, dass ein Zeichen aus dieser Auswahl dann an dieser Stelle steht. Mithilfe von Quantoren kann definiert werden, wie viele Zeichen einer Auswahl hintereinander stehen dürfen. Das Pluszeichen ist genau so ein Quantor, welcher aussagt, dass mindestens ein oder mehrere Zeichen der Zeichenauswahl hintereinander vorkommen muss.
Das Dollarzeichen definiert das Ende einer Zeichenkette. Dadurch das ein Bindestrich vor das Dollarzeichen des regulären Ausdrucks gesetzt haben, bedeutet der reguläre Teilausdruck _-$_, dass die Zeichenkette auf einem Bindestrich endet. Bei dem |-Zeichen handelt es sich um eine logische Oderverknüpfung, die es ermöglicht, mehrere reguläre Ausdrücke zu verknüpfen. In unserem Fall _\w+_.
Der zweite reguläre Ausdruck _\w+_ definiert alle alphanumerischen Zeichenketten mit einem oder mehreren Elementen als Token. Hierbei ist _\w_ eine vordefinierte Zeichenklasse für den regulären Ausdruck <i>[a-zA-Z_0-9]</i> und beinhaltet außer alphanumerische Werte auch noch den Unterstrich. Das Pluszeichen ist hier wieder der Quantor, welcher aussagt, dass aus dieser Zeichenklasse ein oder mehrere Zeichen hintereinander vorkommen muss.
Mithilfe der _tokenize_-Methode wird der reguläre Ausdruck auf den String _prepText_ angewendet. Jeder Substring des mitgegebenen Strings, der den regulären Ausdruck erfüllt, wird an die Liste _tokenList_ angefügt.

Der Grund warum die Wörter, die auf einem Bindestrich enden, bei der Tokenerzeugen extra beachtet werden, ist der, dass die Wörter, welche bei Zeilenumbrüchen getrennt werden, wieder zusammengefügt werden sollen. In der for-Schleife werden diese Tokens auf die Eigenschaft hin, auf einem Bindestrich zu enden, geprüft und gegebenenfalls zusammengesetzt. Dazu wird der Bindestrich aus dem Token entfernt und mit dem nächsten Token in der Tokenliste verknüpft (_token[:-1]+tokenList[index+1]_). Die Tokenliste wird darauf hin aktualisiert. Der neu zusammengesetzte Token ersetzt den Token mit dem Bindestrich und der nächste Token der Liste wird gelöscht. 

Diese Methode ist jedoch nicht immer korrekt, denn es kann auch folgender Fall eintreten: Eine Zeile endet zum Beispiel mit _Damen-_ und die nächste Zeile geht mit _und Herrenschuhe_ weiter. In diesem Fall ist der Bindestrich gewollt, der Algorithmus fügt jedoch die Wörter _Damen_ und _und_ zu einem Wort zusammen. Da davon auszugehen ist, dass dieser Fall selten eintritt, wurde er aber vernachlässigt.

In [21]:
def _preprocessText(self, text):
    
    lowerText = text.lower()
    
    prepText = re.sub(r'\d+', '', lowerText)
            
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+-$|\w+')
    tokenList = tokenizer.tokenize(prepText)
    
    for token in tokenList:
        if token[-1] == '-':
            
            index = tokenList.index(token)
            compositeWord = token[:-1]+tokenList[index+1]
            tokenList[index] = compositeWord
            del tokenList[index+1]
            
    return tokenList
    
Index._preprocessText = _preprocessText

### retrieve
Die <i>retrieve</i>-Methode dient der Suche nach Dokumenten anhand einer eingegebenen Suchquery und stellt die Schnittstelle zum Nutzer dar. Die Idee dabei ist, dass der Nutzer ein oder mehrere Schlagworte, in Form eines Strings, der Methode übergeben kann. Auf deren Basis werden die gefundenen Dokumente, nach Relevanz sortiert, zurückgeliefert.

Zunächst wird der Suchstring des Nutzers mittels der bereits bekannten Methode <i>\_preprocessText</i> normalisiert und in eine Liste von Termen zerteilt. Das Ergebnis wird in der Variable <i>processedStrings</i> abgespeichert. Im zweiten Schritt werden alle genutzten Variablen deklariert. Mithilfe der For-Schleife wird über die Liste von Termen <i>processedStrings</i> iteriert, um über den invertierten Index für jeden Term die Menge der Document IDs zu beschaffen. Desweiteren wird für jeden Term die Länge seiner Menge von Document IDs im Dictionary <i>df</i> abgespeichert und repräsentiert so die Document Frequency. Die Variable <i>result</i> stellt die Menge da, die alle zur Suchquery gefundenen Document IDs enthält. So wird diese Variable mit jeder Menge von Document IDs vereinigt. Da es sich hier um eine Menge handelt kann ausgeschlossen werden, dass Document IDs doppelt vorkommen. Existiert der Term <i>word</i> nicht als Key im Index, da der Nutzer ein Wort eingegeben hat, welches kein Dokument beinhaltet, wird beim nächsten Term der Liste <i>processedStrings</i> fortgefahren.

In der nächsten For-Schleife wird für jede Document ID in der Liste <i>result</i>, mithilfe der <i>docHashmap</i>, die passende Documentinstanz geholt, um für jedes Dokument das TF-IDF-Maß auszurechnen. Wurde dies mithilfe der <i>tf_idf</i>-Methode getan, wird das Gewicht zu der jeweiligen Document ID in dem Dictionary <i>weightedDocs</i> gespeichert. Dieses wird mit der <i>sorted</i>-Funktion von Python nach den Gewichten sortiert. Die Sortierung ist jedoch
aufsteigend, obwohl es gewünscht ist, die Dokumente mit einem höheren Gewicht oben zu haben. Dafür wird die Liste am Ende einfach invertiert.

In der letzten For-Schleife werden die absoluten Pfade (<i>ind.docHashmap[Key].url</i>), in der sortierten Reihenfolge, in die Liste <i>resultList</i> gespeichert. Diese wird dann invertiert, wegen des oben genannten Grundes, und zurückgegeben.

In [22]:
def retrieve(self, searchString):
 
    processedStrings = self._preprocessText(searchString)
    result = set()
    df, weightedDocs = {}, {}
    resultList = []
    
    for word in processedStrings:
        try:
            documents = set(self.invIndex[word])
            df[word] = len(documents)
            result = result.union(documents)
            
        except KeyError:
            continue

    for document in result:
        doc = ind.docHashmap[document]
        doc.tf_idf(processedStrings, df, self.fileCount)
        weightedDocs[doc.docId] = doc.score
        
    sortedDocs = sorted(weightedDocs.items(), key=operator.itemgetter(1))
    
    for key,_ in sortedDocs:
        resultList.append(ind.docHashmap[key].url)
        
    return resultList[::-1]

Index.retrieve = retrieve

## Ausführung der Suchmaschine
In diesem Abschnitt soll kurz erläutert werden, wie die oben erstellten Klassen genutzt werden. Als erstes muss eine neue Instanz der Indexklasse erstellt werden. Über diese Instanz
wird dann die <i>buildIndex</i>-Methode aufgerufen. Optional kann hier eine Liste der gewünschten Startverzeichnisse als raw Strings übergeben werden. Wenn der invertierte Index aufgebaut wurde, kann über die Indexinstanz die <i>retrieve</i>-Methode mit der gewünschten Suchquery aufgerufen werden. Diese liefert eine nach Gewichten sortierte Liste, mit den Pfaden zu den gefundenen Dokumenten. Wenn diese nicht leer ist, können mithilfe einer For-Schleife die Elemente ausgegeben werden.

In [11]:
ind = Index()
ind.buildIndex([r"C:\Users\marle\OneDrive\Studium"])

In [12]:
resultSet = ind.retrieve("Jesse Richter Java")
if resultSet:
    for elem in resultSet:
        print(elem)

C:\Users\marle\OneDrive\Studium\Software Engineering 2\2_DesignPatterns-MusterKlassifikation.pdf
C:\Users\marle\OneDrive\Studium\Software Architecture Management\SoftwareArchitektur_folien.pdf
C:\Users\marle\OneDrive\Studium\Software Architecture Management\main.pdf
C:\Users\marle\OneDrive\Studium\Qualitäts-Sicherung\Softwarequalitätssicherung_Teil_I_2017.pdf
C:\Users\marle\OneDrive\Studium\Praxisbericht_3\Bewertung\Drucken.pdf
C:\Users\marle\OneDrive\Studium\Praxisbericht_3\Bewertung\Ablauf_und_Reflexion_der_Praxisphase_Teil_B.pdf
C:\Users\marle\OneDrive\Studium\Praxisbericht_3\T3000_TINF16AIBI_Richter_Jesse-Jermaine.pdf
C:\Users\marle\OneDrive\Studium\Praxisbericht_2\Scan_JesseRichter_20180914124926.pdf
C:\Users\marle\OneDrive\Studium\Praxisbericht_2\Monitoring.pdf
C:\Users\marle\OneDrive\Studium\Praxisbericht_2\Monitoring-LAPTOP-2NADN4MG.pdf
C:\Users\marle\OneDrive\Studium\Praxisbericht_2\Monitoring-LAPTOP-2NADN4MG-2.pdf
C:\Users\marle\OneDrive\Studium\Logik\logic.pdf
C:\Users\marle